LE QUERNEC Loévan, EL HACHEM Gabriel, and FALCK Jade, DIA2

# BI & DataViz / Machine learning for CV project: Human Action Recognition (HAR)

In [1]:
import warnings
warnings.filterwarnings("ignore")

## Illustration of the problem

## Data visualization

In [2]:
# define the paths
train_dir = 'train'
test_dir = 'test'
train_csv = 'training_set.csv' # file with labeled data
test_csv = 'testing_set.csv' # file with unlabeled data

In [3]:
import pandas as pd

# load the data
train_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

In [ ]:
# count the occurrences of each label
print("Number of lines in the training set:", len(train_df), "\n")
print(train_df['label'].value_counts())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# visualize the distribution of the labels
plt.figure(figsize=(12, 6))
sns.countplot(y='label', data=train_df, order=train_df['label'].value_counts().index)
plt.title('Distribution of the classes in the training set')
plt.xlabel('Number of occurrences')
plt.ylabel('Classes')
plt.show()

What we see is that the data is perfectly balanced, which is a good thing for the training of our model. Do a visualization of the data to see the distribution of the classes is then not necessary.

## Methodology

## Computational results

### Extraction of caracteristics

Now we will encode the labels of the actions to integers. This will allow us to use them in the model. We will be able to retrieve the original labels with the variable `label_mapping`.

In [6]:
# encode the labels
label_mapping = {label: idx for idx, label in enumerate(train_df['label'].unique())}
train_df['label_idx'] = train_df['label'].map(label_mapping)

In [ ]:
print(label_mapping)

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array

img_width, img_height = 128, 128 # dimensions of our images

# prepare the training data
def load_images_from_dataframe(df, directory, img_width, img_height):
    images = []
    labels = []
    for _, row in df.iterrows():
        img_path = os.path.join(directory, row['filename'])
        print(img_path)
        if os.path.exists(img_path):
            # load and rescale the image
            img = load_img(img_path, target_size=(img_width, img_height))
            img_array = img_to_array(img) / 255.0 # normalize
            images.append(img_array)
            if 'label_idx' in row:
                labels.append(row['label_idx'])
    return np.array(images), np.array(labels)

In [ ]:
from tensorflow.keras.utils import to_categorical

# load the traing images
X_train, y_train = load_images_from_dataframe(train_df, train_dir, img_width, img_height)
y_train = to_categorical(y_train, num_classes=len(label_mapping))

In [ ]:
# load the test images
X_test, _ = load_images_from_dataframe(test_df, test_dir, img_width, img_height)

### Training

Specify the model(s) you want to launch the training and create a model file:

In [11]:
cnn = 0
mobilenet = 0

# hyperparameters
batch_size = 32
epochs = 10

In [12]:
if cnn:
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

    # build a simple CNN
    cnn_model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(len(label_mapping), activation='softmax')
    ])

    # compile the model
    cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # train the model
    cnn_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

    # save the model
    cnn_model.save('cnn_model.h5')

In [13]:
if mobilenet:
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.models import Model
    from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

    # load the base model MobileNetV2 pre-trained
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(len(label_mapping), activation='softmax')(x)

    # create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    # compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    # train the model
    model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)

    # save the model
    model.save('mobilenet_model.h5')

### Predictions

Select the model you want to use to make the predictions and have the results:

In [ ]:
import tensorflow as tf

# load the model
cnn_model = tf.keras.models.load_model(f'cnn_model.h5')
mobilenet_model = tf.keras.models.load_model(f'mobilenet_model.h5')

In [ ]:
# make the predictions
print("Train predictions on the CNN model:")
cnn_train_predictions = cnn_model.predict(X_train)
cnn_train_predicted_labels = [list(label_mapping.keys())[np.argmax(pred)] for pred in cnn_train_predictions]

print("Test predictions on the CNN model:")
cnn_test_predictions = cnn_model.predict(X_test)
cnn_test_predicted_labels = [list(label_mapping.keys())[np.argmax(pred)] for pred in cnn_test_predictions]

In [ ]:
# make the predictions
print("Train predictions on the MobileNet model:")
mobilenet_train_predictions = mobilenet_model.predict(X_train)
mobilenet_train_predicted_labels = [list(label_mapping.keys())[np.argmax(pred)] for pred in mobilenet_train_predictions]

print("Test predictions on the MobileNet model:")
mobilenet_test_predictions = mobilenet_model.predict(X_test)
mobilenet_test_predicted_labels = [list(label_mapping.keys())[np.argmax(pred)] for pred in mobilenet_test_predictions]

### Results

In [ ]:
# calculate the accuracy on the training set
true_train_labels = [list(label_mapping.keys())[label] for label in np.argmax(y_train, axis=1)]

cnn_correct_predictions = sum(pred == true for pred, true in zip(cnn_train_predicted_labels, true_train_labels))
cnn_train_accuracy = cnn_correct_predictions / len(true_train_labels) * 100

print(f"CNN accuracy on the training set: {cnn_train_accuracy:.2f}%")

mobilenet_correct_predictions = sum(pred == true for pred, true in zip(mobilenet_train_predicted_labels, true_train_labels))
mobilenet_train_accuracy = mobilenet_correct_predictions / len(true_train_labels) * 100

print(f"MobileNet accuracy on the training set: {mobilenet_train_accuracy:.2f}%")

In [ ]:
# accuracies
accuracies = {
    'CNN': cnn_train_accuracy,
    'MobileNet': mobilenet_train_accuracy
}

# plot the accuracies
plt.figure(figsize=(10, 6))
plt.bar(accuracies.keys(), accuracies.values(), color=['blue', 'green'])
plt.xlabel('Model')
plt.ylabel('Accuracy (%)')
plt.title('Training Accuracy of Different Models')
plt.ylim(0, 100)
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

# function to plot heatmap of confusion matrix
def plot_confusion_matrix(true_labels, predicted_labels, model_name):
    cm = confusion_matrix(true_labels, predicted_labels, labels=list(label_mapping.keys()))
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=list(label_mapping.keys()), yticklabels=list(label_mapping.keys()))
    plt.xlabel('Predicted labels')
    plt.ylabel('True labels')
    plt.title(f'Confusion matrix for {model_name}')
    plt.show()

# plot for CNN model
plot_confusion_matrix(true_train_labels, cnn_train_predicted_labels, 'CNN model')

# plot for MobileNet model
plot_confusion_matrix(true_train_labels, mobilenet_train_predicted_labels, 'MobileNet model')

In [20]:
# function to display images with predictions and true labels
def display_images_with_predictions_and_true_labels(images, predictions, true_labels, num_images=10):
    # limit the number of images
    num_images = min(num_images, len(images))
    
    plt.figure(figsize=(15, 10))
    
    for i in range(num_images):
        plt.subplot(2, 5, i + 1) # 2 rows, 5 columns
        plt.imshow(images[i])
        plt.axis('off')
        
        # compare the prediction with the true label
        predicted_label = predictions[i]
        true_label = true_labels[i]
        color = "green" if predicted_label == true_label else "red"
        
        # add the title and the color
        plt.title(f"Predicted: {predicted_label}\nTrue: {true_label}", color=color, fontsize=12)
    
    plt.tight_layout()
    plt.show()

In [ ]:
X_train_display = (X_train * 255).astype(np.uint8) # convert back to 0-255 range

# print the first 10 images with predictions and true labels
display_images_with_predictions_and_true_labels(
    images=X_train_display[:10],
    predictions=cnn_train_predicted_labels[:10],
    true_labels=true_train_labels
)

In [22]:
# function to display images with predictions
def display_images_with_predictions(images, predictions, num_images=10):
    # limit the number of images
    num_images = min(num_images, len(images))
    
    plt.figure(figsize=(15, 10))
    
    for i in range(num_images):
        plt.subplot(2, 5, i + 1) # 2 rows, 5 columns
        plt.imshow(images[i])
        plt.axis('off')

        plt.title(f"Predicted: {predictions[i]}", fontsize=12)
    
    plt.tight_layout()
    plt.show()

In [ ]:
X_test_display = (X_test * 255).astype(np.uint8) # convert back to 0-255 range

# print the first 10 images with predictions
display_images_with_predictions(
    images=X_test_display[:10],
    predictions=cnn_test_predicted_labels[:10]
)

In [41]:
def get_top_channels_by_variance(activation_maps, top_k=8):
    # check the dimensions of the activation maps
    if len(activation_maps.shape) == 3: # single image (height, width, channels)
        channel_variances = np.var(activation_maps, axis=(0, 1))
    elif len(activation_maps.shape) == 4: # batch of images (batch, height, width, channels)
        channel_variances = np.var(activation_maps, axis=(0, 1, 2))
    else:
        raise ValueError("Unexpected dimensions for activation_maps: {}".format(activation_maps.shape))
    
    # find the indices of the top-k channels with highest variance
    top_channels = np.argsort(channel_variances)[-top_k:]

    return top_channels

In [42]:
def get_top_channels_by_mean(activation_maps, top_k=8):
    # check the dimensions of the activation maps
    if len(activation_maps.shape) == 2: # (positions, channels)
        channel_means = np.mean(activation_maps, axis=0)
    elif len(activation_maps.shape) == 3: # (height, width, channels)
        channel_means = np.mean(activation_maps, axis=(0, 1))
    else:
        raise ValueError(f"Unexpected dimensions for activation_maps: {activation_maps.shape}")
    
    # find the indices of the top-k channels with highest mean
    top_channels = np.argsort(channel_means)[-top_k:]

    return top_channels

In [43]:
def get_top_channels_by_max(activation_maps, top_k=8):
    # check the dimensions of the activation maps
    if len(activation_maps.shape) == 2: # (positions, channels)
        channel_max = np.max(activation_maps, axis=0)
    elif len(activation_maps.shape) == 3: # (height, width, channels)
        channel_max = np.max(activation_maps, axis=(0, 1))
    else:
        raise ValueError(f"Unexpected dimensions for activation_maps: {activation_maps.shape}")
    
    # find the indices of the top-k channels with highest max
    top_channels = np.argsort(channel_max)[-top_k:]

    return top_channels

In [44]:
def display_important_activation_maps(activation_maps, image, important_channels, max_channels=8):
    # limit the number of channels
    important_channels = important_channels[:max_channels]
    
    # number of important channels
    num_channels = len(important_channels)
    fig, axes = plt.subplots(1, num_channels + 1, figsize=(20, 5))
    
    # display the original image
    axes[0].imshow(image.astype("uint8"))
    axes[0].set_title("Image originale")
    axes[0].axis('off')
    
    # display the important channels
    for i, channel in enumerate(important_channels):
        activation_map = activation_maps[:, :, channel]
        axes[i + 1].imshow(activation_map, cmap='viridis')
        axes[i + 1].set_title(f"Canal {channel}")
        axes[i + 1].axis('off')
    
    plt.tight_layout()
    plt.show()

In [45]:
from tensorflow.keras.models import Model

def get_activation_maps(model, img_array, layer_name):
    # create a model that will return these outputs, given the model input
    intermediate_layer_model = Model(inputs=model.input, outputs=model.get_layer(layer_name).output)
    intermediate_output = intermediate_layer_model.predict(img_array)
    
    # check the dimensions of the intermediate output
    if len(intermediate_output.shape) == 4: # (batch, height, width, channels)
        return intermediate_output
    else:
        raise ValueError("Unexpected dimensions for activation maps.")

In [ ]:
# get the activation maps for the first image
activation_maps = get_activation_maps(cnn_model, X_test, 'conv2d_2')

In [ ]:
# get the top channels by variance
top_channels_variance = get_top_channels_by_variance(activation_maps[0], top_k=8)

# display the important activation maps
display_important_activation_maps(
    activation_maps=activation_maps[0],
    image=X_test_display[0],
    important_channels=top_channels_variance,
    max_channels=8
)

In [ ]:
# get the top channels by mean
top_channels_mean = get_top_channels_by_mean(activation_maps[0], top_k=8)

# display the important activation maps
display_important_activation_maps(
    activation_maps=activation_maps[0],
    image=X_test_display[0],
    important_channels=top_channels_mean,
    max_channels=8
)

In [ ]:
# get the top channels by max
top_channels_max = get_top_channels_by_max(activation_maps[0], top_k=8)

# display the important activation maps
display_important_activation_maps(
    activation_maps=activation_maps[0],
    image=X_test_display[0],
    important_channels=top_channels_max,
    max_channels=8
)

In [ ]:
from sklearn.manifold import TSNE

# apply TSNE on the CNN model predictions
tsne = TSNE(n_components=2, random_state=42)
cnn_tsne_results = tsne.fit_transform(cnn_train_predictions)

# apply TSNE on the MobileNet model predictions
mobilenet_tsne_results = tsne.fit_transform(mobilenet_train_predictions)

# plot for CNN model
plt.figure(figsize=(12, 6))
sns.scatterplot(x=cnn_tsne_results[:, 0], y=cnn_tsne_results[:, 1], hue=true_train_labels, palette='tab10', legend='full')
plt.title('TSNE visualization of CNN model predictions')
plt.xlabel('TSNE component 1')
plt.ylabel('TSNE component 2')
plt.show()

# plot for MobileNet model
plt.figure(figsize=(12, 6))
sns.scatterplot(x=mobilenet_tsne_results[:, 0], y=mobilenet_tsne_results[:, 1], hue=true_train_labels, palette='tab10', legend='full')
plt.title('TSNE visualization of MobileNet model predictions')
plt.xlabel('TSNE component 1')
plt.ylabel('TSNE component 2')
plt.show()

## Analysis and interpretation